# Introduction to SEUIF97 Library

 SEUIF97水和水蒸汽物性计算库是以IAPWS IF97公式为标准，C语言实现的高速，多平台适用共享库。
 
 Windows 32/64位版: 从 https://github.com/SEU03013050C/SEUIF97 下载：SEUIF97.dll和 seuif97.py，
 
 然后：
 
   1) SEUIF97.dll拷贝到 c:\windows\system
   
   2) seuif97.py拷贝到 c:\python34\Lib
   
 
SEUIF97仓库中同时有Linux共享库及Python接口模块。

# SEUIF97共享库接口简介

1 接口调用约定：

       Windows：stdcall,可被VBA等调用

       Linux:  cdecl
      
 2 接口函数约定： 

 IAPWS-IF97 水和水蒸汽物性计算软件包提供物性计算和热力过程计算两类函数：
 
 <b>物性计算函数基本形式</b>:if97inAinB( inA,inB, propertyID)
    
      inA -  双精度型，第一已知物性， 可以是 p,t,h
      
      inB -  双精度型，第二已知物性， 可以是 t,h,s,v,x
      
      propertyID  - 整型，为需要计算物性的编码，从0到29共30个。（请参考“ SEUIF97物性列表.doc）
  
      如已知（p,t)求其他物性的函数头定义：  
      
           Windows中使用的stdcall调用约定：
        
               extern  "C" __stdcall __declspec(dllexport)  double seupt(double p,double t,int wp);   
      
           Linux下使用的cdecl调用约定：
      
               extern  "C"  double seupt(double p,double t,int wp);
      
      
<ol>
<li>if97pt ( pressure, temperature, propertyID )
<li>if97ph (pressure, enthalpy, propertyID )
<li>if97ps ( pressure, entropy, propertyID )
<li>if97pv ( pressure , volume, property ID)
<li>if97th ( temperature, enthalpy, propertyID )
<li>if97ts ( temperature, entropy, property ID)
<li>if97tv ( temperature , volume, property ID)
<li>if97hs ( enthalpy , entropy, property ID)
<li>if97px ( pressure, quality, propertyID )
<li>if97tx ( temperature , quality, propertyID )
</ol>

<p><b>热力过程</b>

     1 等熵焓降： ishd( pi,ti,pe)
   
        pi - 双精度型，入口压力； ti - 双精度型，入口温度
        
        pe - 双精度型，出口压力

   2 内效率： ief( pi,ti,pe,te)
        
        pi  - 双精度型，入口压力；ti  - 双精度型，入口温度
        
        pe  - 双精度型，出口压力；te  - 双精度型， 出口温度



# Python 接口：seuif97.py

使用Python标准库ctypes调用、封装共享库

In [ ]:
# -*- coding: utf-8 -*- 

from ctypes import c_int,c_double,WINFUNCTYPE,windll
prototype=WINFUNCTYPE(c_double,c_double,c_double,c_int)

fpt=prototype(("seupt",windll.libseuif97),)

def pt(p,t,w): 
    result=fpt(p,t,w)
    return result

def pt2h(p,t): 
    result=fpt(p,t,4)
    return result

# 示例1： 简单调用

In [ ]:
from seuif97 import *
p=16.12
t=562.1
h=pt(p,t,4)
s=pt(p,t,5)
v=pt2v(p,t)
print('h: ',h,' s: ',s,' v: ',v)
t1=ph(p,h,1)
t2=ps(p,s,1)
t3=pv(p,v,1)
print('t1: ',t1,' t2 ',t2,' t3: ',t3)
p1=th(t,h,0)
p2=ts(t,s,0)
p3=tv(t,v,0)
print('p1: ',p1,' p2 ',p2,' p3: ',p3)
# %timeit p1=th(t,h,0)

# 示例2： 朗肯循环效率

<p>朗肯循环热效率计算：</p>
<p>循环定压加热量：${q_1=h_1'-h_5 (kJ/kg)}$ </p>
<p>循环定压加热量：${q_1=h_1'-h_5 (kJ/kg)}$ </p>

<p>算例：华能玉环 USC 26.25/600/600 ，设计：发电效率44.86%，计算其理想朗肯循环热效率

In [3]:
from seuif97 import *
p1=26.25
t1=600
p2=4.8*0.001

# 1 汽轮机定熵膨胀作功：h1-h2

# 1.1 计算h1
h1=pt2h(p1,t1)

# 1.2 计算h2
s1=pt(p1,t1,5) # s2=s1  定熵膨胀
s2=s1
s2w=px2s(p2,0)
s2s=px2s(p2,1)
x2=(s2-s2w)/(s2s-s2w) 

h2w=px2h(p2,0)
h2s=px2h(p2,1)
h2=h2w+x2*(h2s-h2w)

w=h1-h2
print('汽轮机定熵膨胀作功=',w,' kJ/kg')

# 给水泵耗功：h5-h3

p3=p2 # 定压 
t3=px2t(p3,0)
s3 =px2s(p3,0) #s3w
h3 =px2h(p3,0) #h3w

p5=p1 # 定压
s5=s3
h5=ps2h(p5,s5)

wpump=h5-h3 #给水泵定熵压缩
wq=w-wpump

q1=h1-h5 # 定压加热
print('定压加热=',q1,' kJ/kg')  

ef=100*wq/q1
print('理想热效率=',ef,'%')                                                     

汽轮机定熵膨胀作功= 1556.9218216924253  kJ/kg
定压加热= 3321.1304584876  kJ/kg
理想热效率= 46.08944674210697 %


# 热力循环极限潜力：卡诺循环

In [4]:
T1=30+273.15
T2=600+273.15  # 华能玉环 USC 26.25/600/600  发电效率44.86%
e=1-T1/T2
print('ef=',e*100)

ef= 65.28087957395637


#  示例3 内效率

In [6]:
from seuif97 import *

# HP of 300MW Steam Turbine 

p1=16
t1=535
p2=3.56
t2=315

# 使用过程方法
hdis=ishd(p1,t1,p2) # 等熵焓降
ef=ief(p1,t1,p2,t2) # 过程效率
print('等熵焓降 =',hdis)
print('过程效率 =',ef)

等熵焓降 = 425.003134565186
过程效率 = 89.92487193794696
